In [1]:
import pymolpro
import random
import numpy as np
import pandas as pd
import os
import math
def chkorbid(o1,o2):
    if o1.startswith('-'):
        chko1=o1[1:]
    else:
        chko1=o1
    if o2.startswith('-'):
        chko2=o2[1:]
    else:
        chko2=o2
    if chko1==chko2:
        return True
    else: 
        return False

def distances(o1,o2,a):
    #the dictionary only contains +ve orbital IDs so remove
    if o1.startswith('-'):
        chko1=o1[1:]
    else:
        chko1=o1
    if o2.startswith('-'):
        chko2=o2[1:]
    else:
        chko2=o2
    big_list=[]
    for i in grid_dt[chko1]:
        tmp_list=[]
        for j in grid_dt[chko2]:
            #find the distance
            o1x,o1y,o1z = grid_dt[chko1][i]['x'], grid_dt[chko1][i]['y'],grid_dt[chko1][i]['z']
            o2x,o2y,o2z = grid_dt[chko2][j]['x'], grid_dt[chko2][j]['y'],grid_dt[chko2][j]['z']
            r=math.sqrt((o1x-o2x)**2+(o1y-o2y)**2+(o1z-o2z)**2)
            #correlation energy should scale like r^-6 so take the reciprocal, going to lead to problems at 0 so take an error function
            if r==0:
                w=a
            else:   
                w=math.erf(a*r)/r
            big_list.append(w)
            if j==i:
                break
        #big_list.append(tmp_list)
    return big_list

def orbitalsize(orbital):
    return math.sqrt(sum([orbital.local_second_moments[i][i] for i in range(3)]))

def array_former(distance_list,features_list):
    
    zero_array=np.zeros((len(distance_list)-1,len(features_list)))
    feature_array=np.array(features_list)
    feature_array=np.reshape(feature_array,(1,len(features_list)))
    ft=np.vstack((features_list,zero_array))
    distance_array=np.array(distance_list)
    distance_array=np.reshape(distance_array,(len(distance_list),1))
    return np.hstack((distance_array,ft))
    

In [2]:
database=pymolpro.database.run('Bak2000_atomisations','locali,pipek;mp2','aug-cc-pvtz',postamble='put,xml', backend='local')

In [3]:
#settings
#grid dimensions
n=4
#alpha in the error function
a=1

q=0
for k,v in database.projects.items():
    orbital_dt={}
    for orb in v.orbitals(minocc=1.00):
        points=orb.grid(4)
        
        values=orb.evaluate(points)
        grid_dt.update({orbital.ID:{i:{points[i][0],points[i][1],points[i][2],values[i]}}for i in range(len(points))})
        orbital_dt.update({orbital.ID:{
        'energy':p.xpath("//orbitals/orbital[@ID='"+orbital.ID+"']/@energy"),
        'size':orbitalsize(orbital)
        }})
        oid=f'-{orbital.ID}'
        
        orbital_dt.update({oid:{
        'energy':p.xpath("//orbitals/orbital[@ID='"+orbital.ID+"']/@energy"),
        'size':orbitalsize(orbital)
        }})

print(grid_dt[0])


Exception: something is wrong: there should be just one orbital set